In [0]:
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *

**Q1. While ingesting customer data from an external source, you notice duplicate entries. How would you remove duplicates and retain only the latest entry based on a timestamp column?**

In [0]:
data = [("101","2023-12-01",100),("101","2023-12-02",150),
        ("102","2023-12-01",200),("102","2023-12-02",250)]

schema = ["product_id","date","sales"]

sales_df = spark.createDataFrame(data,schema)

sales_df = sales_df.withColumn("date",col("date").cast(DateType()))

# SOLUTION-1
window = Window.partitionBy("product_id").orderBy(desc("date"))
sales_df.withColumn("rank",rank().over(window)).filter(col("rank") == 1).select("product_id","date","sales").display()

# SOLUTION-2
sales_df.sort("product_id","date",ascending=[1,0]).dropDuplicates(subset=["product_id"]).display()

product_id,date,sales
101,2023-12-02,150
102,2023-12-02,250


product_id,date,sales
101,2023-12-02,150
102,2023-12-02,250


**Q2. While processing data from multiple files with inconsistent schemas, you need to merge them on a single dataframe. How would you handle this inconsistency in pyspark?**

In [0]:
# MergeSchema -- Consider all files and create a dataframe on top of all those

df = spark.read.format("csv") \
                .option("header","True") \
                .option("mergeSchema","True") \
                .load("/FileStore/temp/")

df.display()

empid,empname,gender,sal,dept,Hello
1,wafa,male,5000,IT,null
2,maheer,male,3000,Payroll,null
3,asmin,female,3000,Payroll,null
4,sarfarj,male,4000,HR,null
5,ayesha,female,3000,IT,null
6,arfin,female,2500,HR,null
7,shahin,female,4000,IT,null
8,mahaboob,male,2000,Payroll,null
9,pyarijan,female,1500,IT,null
10,imam,male,2000,IT,null


**Q3. You are working with a real-time data pipeline. you notice missing values in that streaming data column - Category. How would you handle those null or missing values in such a scenario?** \
**df_stream = spark.readStream.schema("id INT, value STRING").csv("/path/to/stream")**

In [0]:
result_df = df.fillna({"Hello":"N/A"})
result_df.display()

empid,empname,gender,sal,dept,Hello
1,wafa,male,5000,IT,N/A
2,maheer,male,3000,Payroll,N/A
3,asmin,female,3000,Payroll,N/A
4,sarfarj,male,4000,HR,N/A
5,ayesha,female,3000,IT,N/A
6,arfin,female,2500,HR,N/A
7,shahin,female,4000,IT,N/A
8,mahaboob,male,2000,Payroll,N/A
9,pyarijan,female,1500,IT,N/A
10,imam,male,2000,IT,N/A


**Q4. You need to calculate total number of actions performed by user given in the system. How would you calculate the top 5 most active users?**

In [0]:
data = [("user1",5),("user2",8),("user3",2),("user4",10),("user2",3),("user6",18),("user7",22),("user1",2),("user9",10)]

columns = ["user_id","actions"]

user_df = spark.createDataFrame(data,columns)

total_actions = user_df.groupBy("user_id").agg(sum(col("actions")).alias("Total_Actions"))

total_actions.display()

top_5 = total_actions.sort(col("Total_Actions").desc()).limit(5)

top_5.display()

user_id,Total_Actions
user1,7
user2,11
user3,2
user4,10
user6,18
user7,22
user9,10


user_id,Total_Actions
user7,22
user6,18
user2,11
user4,10
user9,10


**Q5. While processing sales transaction data, you need to identify the most recent transaction for each customer. How would you approach to this task?**

In [0]:
data = [("cust1","2023-12-01",100),("cust2","2023-12-02",150),
        ("cust1","2023-12-03",200),("cust2","2023-12-04",250)]

columns = ["cust_id","transaction_date","sales"]

cust_sales_df = spark.createDataFrame(data,columns)

cust_sales_df.withColumn("transaction_date",to_date(col("transaction_date"))) \
             .withColumn("recent_txn",dense_rank().over(Window.partitionBy("cust_id").orderBy(desc("transaction_date")))) \
             .filter(col("recent_txn") == 1).select("cust_id","transaction_date","sales").display()

cust_id,transaction_date,sales
cust1,2023-12-03,200
cust2,2023-12-04,250


**Q6. You need to identify customers who haven't made any purchase in the last 30 days. How would you filter such customers?**

In [0]:
data = [("cust1","2025-03-01"),("cust2","2024-11-20"),("cust3","2024-11-25")]

columns = ["cust_id","last_purchase_date"]

cust_pur_df = spark.createDataFrame(data,columns)

cust_pur_df.withColumn("last_purchase_date",to_date(col("last_purchase_date"))) \
           .filter(datediff(current_date(),col("last_purchase_date")) > 30).display() 

cust_id,last_purchase_date
cust2,2024-11-20
cust3,2024-11-25


**Q7. While analyzing customer reviews, you need to identify most frequently used words in the feedback. How would you implement this?**

In [0]:
data = [("cust1","The product is great"),("cust2","Great product, fast delivery"),("cust3","not bad, could be better")]

columns = ["cust_id","feedback"]

cust_fdbck_df = spark.createDataFrame(data,columns)

word_feedback = cust_fdbck_df.withColumn("feedback",explode(split(col("feedback")," "))).withColumn("feedback",lower(col("feedback")))

word_feedback.groupBy("feedback").agg(count(col("feedback")).alias("word_count")).display()

feedback,word_count
great,2
is,1
the,1
product,1
fast,1
delivery,1
"product,",1
could,1
not,1
be,1


**Q8. You need to calculate cumulative sum of sales over time for each product. How would you approach this?**

In [0]:
data = [("product1","2023-12-01",100),("product2","2023-12-02",200),
        ("product1","2023-12-03",150),("product2","2023-12-04",250)]

columns = ["product_id","date","sales"]

prd_df = spark.createDataFrame(data,columns)

prd_df.withColumn("date",to_date(col("date"))) \
      .withColumn("cumsum",sum("sales").over(Window.partitionBy("product_id").orderBy("date"))) \
      .display()

product_id,date,sales,cumsum
product1,2023-12-01,100,100
product1,2023-12-03,150,250
product2,2023-12-02,200,200
product2,2023-12-04,250,450


**Q9. While preparing a data pipeline, you notice some duplicate rows in the dataset. How would you remove duplicates without affecting the original order?**


In [0]:
data = [("John",25),("Jane",30),("John",25),("Alice",22)]

columns = ["name","age"]

data_df = spark.createDataFrame(data,columns)

data_df.withColumn("r_no",row_number().over(Window.partitionBy("name").orderBy("age"))).filter(col("r_no")==1).display()

name,age,r_no
Alice,22,1
Jane,30,1
John,25,1


**Q10. You are working with user activity data and need to average session duration per user. How would you implement this?**

In [0]:
data = [("user1","2023-12-01",50),("user1","2023-12-02",60),
        ("user2","2023-12-01",45),("user2","2023-12-03",75)]

columns = ["user_id","session_date","duration"]

df = spark.createDataFrame(data,columns)

df.groupBy("user_id").agg(avg("duration").alias("Avg_duration")).display()

user_id,Avg_duration
user1,55.0
user2,60.0


**Q11. While analysing sales data, you need to find out the product with the highest sales for each month. How would you approach to this task?**

In [0]:
data = [("product1","2023-12-01",100),("product2","2023-12-01",150),
        ("product1","2023-12-02",200),("product2","2023-12-02",250)]

columns = ["product_id","date","sales"]

prd_sales_df = spark.createDataFrame(data,columns)

prd_sales_df.withColumn("date",to_date(col("date"))) \
                            .withColumn("month",month(col("date"))) \
                            .groupBy("month","product_id").agg(sum("sales").alias("Total_monthly_sales")) \
                            .withColumn("rank_sales",dense_rank().over(Window.partitionBy("month").orderBy(desc("Total_monthly_sales")))) \
                            .filter(col("rank_sales") == 1) \
                            .select("product_id","month","Total_monthly_sales") \
                            .display()

product_id,month,Total_monthly_sales
product2,12,400


**Q12: You are working with a large delta table that is frequently updated by multiple users. The data is stored in partitions, and sometimes updates can cause inconsistent read due to concurrent transactions. How would you ensure ACID compliance and avoid data corruption in pyspark?**

ENSURE ACID: During Delta table creation it'll auto create delta log, which takes care for ACID compliance.

HANDLE DATA CORRUPTION -- Using SCD-1 or merge or upsert

USING FILE PATH DELTA TABLE

In [0]:
old_data = [("user1",100),("user2",200)]

# create a dataframe
old_df = spark.createDataFrame(old_data, ["user","sales"])

# create a delta table -- first Time
## old_df.write.format("delta").save("/FileStore/delta/Users")

# new data
new_data = [("user3",500),("user2",400)]

new_df = spark.createDataFrame(new_data, ["user","sales"])

from delta.tables import DeltaTable

# create a object of delta table to do advanced operation on the table
delta_tbl = DeltaTable.forPath(spark, "/FileStore/delta/Users")

# Perform upsert/merge on top of delta_table to avoid data corruption
delta_tbl.alias("target").merge(new_df.alias("source"), "target.user = source.user") \
                        .whenMatchedUpdate(set = {"sales": "source.sales"}) \
                        .whenNotMatchedInsert(values = {"user":"source.user",
                                                        "sales":"source.sales"}) \
                        .execute()

# Read delta table data
result_df = spark.read.format("delta").load("/FileStore/delta/Users")

result_df.display()

user,sales
user1,100
user3,500
user2,400


USING MANAGED DELTA TABLE WHICH STORED IN METASTORE AND CAN SEE OUTPUT USING SQL QUERY

In [0]:
old_data = [("cust1","2023-12-01",100),("cust2","2023-11-05",200)]

# create a dataframe
old_df = spark.createDataFrame(old_data, ["cust_id","sales_date","sales"])

# create a delta table -- first Time
## old_df.write.format("delta").saveAsTable("customer_table")

new_data = [("cust1","2023-12-05",590),("cust3","2024-11-10",4000)]

# create a dataframe
new_df = spark.createDataFrame(new_data, ["cust_id","sales_date","sales"])

from delta.tables import DeltaTable

delta_tbl = DeltaTable.forName(spark, "customer_table")

delta_tbl.alias("Target") \
        .merge(new_df.alias("Source"), "Source.cust_id == Target.cust_id") \
        .whenMatchedUpdate(
            set = {
                "sales_date" : "Source.sales_date",
                "sales" : "Source.sales"
            }
        ) \
        .whenNotMatchedInsert(
            values = {
                "cust_id" : "Source.cust_id",
                "sales_date" : "Source.sales_date",
                "sales" : "Source.sales"
            }
        ) \
        .execute()

**Q13: You need to process large dataset stored in parquet format and ensure that all columns have the right schema(Almost).How would you do this?**

In [0]:
df_13 = spark.read.format("parquet") \
                  .option("header","true") \
                  .option("inferSchema","true") \
                .load("/FileStore/tables/part_r_00000_1a9822ba_b8fb_4d8e_844a_ea30d0801b9e_gz.parquet")

df_13.display()

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44


**Q14: You are reading a csv file and need to handle corrupt records gracefully by skipping them.How would you do this?**

In [0]:
corrupt_csv_df = spark.read.format("csv") \
                            .option("header","true") \
                            .option("inferSchema","true") \
                            .option("mode","DROPMALFORMED") \
                            .load("/FileStore/tables/Emp_pysp_details-1.csv")

corrupt_csv_df.display()

id,name,age,salary,address,nominee
1,Manish,26,75000,bihar,n1
2,Nikita,23,100000,up,n2


**Q15. You have a dataset containing names of employees and their departments.You need to find department with most employees.**

In [0]:
data = [("Alice","HR"),("Bob","Finance"),("Charlie","HR"),("David","Engineering"),("Eve","Finance")]

schema = ["emp","dept"]

emp_dept_df = spark.createDataFrame(data,schema)

emp_dept_df.groupBy("dept").agg(count("emp").alias("employee_count")).sort(col("employee_count").desc()).display()

dept,employee_count
HR,2
Finance,2
Engineering,1


**Q16. While processing sales data, you need to classify each transaction as either "high" or "low" based on it's amount. How would you approach to this task using when condition?**

In [0]:
data = [("product1",100),("product2",300),("product3",50)]

columns = ["product_id","sales"]

prd_txn_df = spark.createDataFrame(data,columns)

prd_txn_df.withColumn("txn_category", when(col("sales") > 50 , "high")
                                    .otherwise("low"))\
          .display()

product_id,sales,txn_category
product1,100,high
product2,300,high
product3,50,low


**Q17. While analysing a large dataset, you need to create a new column that holds a timestamp of when the record was processed.How would you implement this and what would be the best USE CASE?**

In [0]:
data = [("product1",100),("product2",200),("product3",300)]

columns = ["product_id","sales"]

prd_17_df = spark.createDataFrame(data,columns)

prd_17_df.withColumn("processed_time", current_timestamp()) \
        .display()


## USE CASE
## For slowly changing dimension we need the date and time when exactly the data got changed

product_id,sales,processed_time
product1,100,2025-03-25T12:52:32.798+0000
product2,200,2025-03-25T12:52:32.798+0000
product3,300,2025-03-25T12:52:32.798+0000


**Q18. You need to registed this PySpark dataframe as a temporary sql object and run a query on it.How would you implement this ?**

In [0]:
data = [("product1",100),("product2",200),("product3",300)]

columns = ["product_id","sales"]

prd_18_df = spark.createDataFrame(data,columns)

prd_18_df.createOrReplaceTempView("Product_table")

In [0]:
%sql
select * from Product_table

product_id,sales
product1,100
product2,200
product3,300


**Q19. You need to registed this PySpark dataframe as a temporary sql object and run a query on it(FROM DIFFERENT NOTEBOOK AS WELL).How would you implement this ?**

In [0]:
data = [("product1",100),("product2",200),("product3",300)]

columns = ["product_id","sales"]

prd_19_df = spark.createDataFrame(data,columns)

prd_19_df.createOrReplaceGlobalTempView("Product_table_global")

In [0]:
%sql
select * from global_temp.Product_table_global

product_id,sales
product1,100
product2,200
product3,300


**Q20. You need to query data from a PySpark DataFrame using SQL, but the data include a nested structure. How would you flatten the data for easier of use?**

In [0]:
data = [("product1",{"price":100,"quantity":2}),("product2",{"price":200,"quantity":5})]

columns = ["product_id","product_info"]

prd_nested_df = spark.createDataFrame(data,columns)

prd_nested_df.withColumn("price",col("product_info.price")) \
            .withColumn("qty",col("product_info.quantity")) \
            .drop(col("product_info")) \
            .createOrReplaceTempView("flat_prd_table")

In [0]:
%sql
select * from flat_prd_table

product_id,price,qty
product1,100,2
product2,200,5


**Q21. While reading data from Parquet, you need to optimize performance the data based on a column. How would you implement this?**

In [0]:
data = [("cust1",200,"IT"),("cust2",500,"HR")]

schema = ["cust_id","sales","dept"]

df = spark.createDataFrame(data,schema)

df.write.format("parquet").partitionBy("dept").save("/FileStore/parq_data/customer.parquet")

In [0]:
df1 = spark.read.parquet("/FileStore/parq_data/customer.parquet/")
df1.rdd.getNumPartitions()
df1.select(spark_partition_id().alias('partid')).groupBy('partid').count().show()

+------+-----+
|partid|count|
+------+-----+
|     0|    1|
|     1|    1|
+------+-----+



**Q22. You are working with large dataset in Parquet format and you need to ensure that the data is written in an optimized manner with proper compression. How would you implement this?**

In [0]:
data = [("cust1",200,"IT"),("cust2",500,"HR")]

schema = ["cust_id","sales","dept"]

df = spark.createDataFrame(data,schema)

df.write.format("parquet").option("compression","snappy").save("/FileStore/parq_data/customer_compress.parquet")

**Q23. You are processing sales data. Group by product categories and create a list of all product names in each category.**

In [0]:
data = [("Electronics","Laptop"),("Electronics","smartphone"),("Furniture","chair"),("Furniture","Table")]

schema = ["category","product"]

df_23 = spark.createDataFrame(data,schema)

df_23.groupBy("category").agg(collect_list("product").alias("product_list")).display()

category,product_list
Electronics,"List(Laptop, smartphone)"
Furniture,"List(chair, Table)"


**Q24. You are analyzing orders, Group by customer IDs and list all unique product IDs each customer purchased.**

In [0]:
data = [(101,"P001"),(101,"P002"),(102,"P001"),(101,"P001")]

schema = ["customer_id","product_id"]

df_24 = spark.createDataFrame(data,schema)

df_24.groupBy("customer_id").agg(collect_set("product_id").alias("unique_product_list")).display()

customer_id,unique_product_list
101,"List(P002, P001)"
102,List(P001)


**Q25. For customer records, combine first and last names only if email address exists.**

In [0]:
data = [("John","Doe","john.doe@gmail.com"),("Jane","Smith",None)]

schema = ["name","surname","email"]

df_25 = spark.createDataFrame(data,schema)

df_25.withColumn("Full_name",when(col("email").isNotNull(),concat_ws(" ","name","surname"))
                            .otherwise(None)).display()

name,surname,email,Full_name
John,Doe,john.doe@gmail.com,John Doe
Jane,Smith,null,null


**Q26. You have a dataframe containing customer IDs and a list of their purchased product IDs. Calculate the number of products each customer has purchased.**

In [0]:
data = [
    (1,["prod1", "prod2", "prod3"]),
    (2, ["prod4"]),
    (3, ["prod5","prod6"])
]

schema = "customer_id INT, product_ids array<STRING>"

df_26 = spark.createDataFrame(data,schema)

df_26.withColumn("number_of_products",size(col("product_ids"))).display()

customer_id,product_ids,number_of_products
1,"List(prod1, prod2, prod3)",3
2,List(prod4),1
3,"List(prod5, prod6)",2


**Q27. You have employee IDs of varying lengths. Ensure IDs are 6 characters long by padding with leading zeroes.**

In [0]:
data = [
    ("1",),
    ("123",),
    ("45679",)
]

schema = ["customer_id"]

df_27 = spark.createDataFrame(data,schema)

df_27.withColumn("customer_id",lpad(col("customer_id"),6,"0")).display()

customer_id
000001
000123
045679


**Q28. You need to validate phone numbers by checking if they start with '91'**

In [0]:
data = [
    ("9112345677",),
    ("8112459879",),
    ("9123567890",)
]

schema = ["phone_number"]

df_28 = spark.createDataFrame(data,schema)

df_28.filter(substring("phone_number",1,2) == "91").display()

phone_number
9112345677
9123567890


**Q29. You have a dataset with courses taken by students. Calculate the average number of courses per student.**

In [0]:
data = [
    (1, ["Math","Science"]),
    (2, ["History"]),
    (3, ["Art","PE","Biology"])
]

schema = ["student_id","courses"]

df_29 = spark.createDataFrame(data,schema)

df_29.withColumn("courses",size(col("courses"))).agg(avg("courses")).display()

avg(courses)
2.0


**Q30. You have a dataset with primary and secondary contact numbers. Use the primary number if available, otherwise use the secondary contact.**

In [0]:
data = [
    (None,"1234567890"),
    ("9876543210",None),
    ("7891234579",("8963638300"))
]

schema = ["p_c","s_c"]

df_30 = spark.createDataFrame(data,schema)

df_30.select("p_c","s_c",coalesce(col("p_c"),col("s_c")).alias("use_contact")).display()

p_c,s_c,use_contact
null,1234567890,1234567890
9876543210,null,9876543210
7891234579,8963638300,7891234579


**Q31. You are categorizing product codes based on their lengths. If the length is 5, label it as "Standard". Otherwise , label it as "custom".**

In [0]:
data = [
    ("prod1",),
    ("prod234",),
    ("prod9876",)
]

schema = ["product_code"]

df_31 = spark.createDataFrame(data,schema)

df_31.withColumn("label", when(length(col("product_code")) > 5 ,"Standard")
                         .otherwise("Custom")).display()

product_code,label
prod1,Custom
prod234,Standard
prod9876,Standard
